In [1]:
import os
import sys
import urllib.request
import pandas as pd
import numpy as np
import json
from tqdm import tqdm_notebook

In [3]:
df = pd.read_excel('./data/dataplc.xlsx',index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28754 entries, 0 to 38947
Data columns (total 11 columns):
address         28754 non-null object
road_address    28754 non-null object
mapx            28754 non-null int64
mapy            28754 non-null int64
type            28754 non-null object
item            28729 non-null object
name            28754 non-null object
place           22590 non-null object
gu              28754 non-null object
dong            28754 non-null object
road            28754 non-null object
dtypes: int64(2), object(9)
memory usage: 2.6+ MB


In [4]:
# del df['상권업종대분류코드']
# del df['상권업종중분류코드']
# del df['상권업종소분류코드']
# del df['표준산업분류코드']
# del df['시도코드']
# del df['시군구코드']
# del df['행정동코드']
# del df['법정동코드']
# del df['지번코드']
# del df['대지구분코드']
# del df['대지구분명']
# del df['도로명코드']
# del df['건물관리번호']
# del df['mapx']
# del df['mapy']

In [14]:
df.reset_index(inplace=True)
del df['index']

In [15]:
def trans_adds(x,y):
    client_id = "GchMYrs8q79adXEdENbw"
    client_secret = "aspsAFfrKJ"
    url_base = "https://openapi.naver.com/v1/map/reversegeocode?encoding=utf-8&coordType=tm128&query="
    url_xy = str(x) + ',' + str(y)
    url = url_base + url_xy
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    response_body = response.read()
    list = json.loads(response_body)
    return list['result']['items'][0]['address']

In [16]:
trans_adds(df['mapx'][69],df['mapy'][69])

'서울특별시 중구 명동2가 81-1'

In [17]:
def trans_coords(add):
    client_id = "GchMYrs8q79adXEdENbw"
    client_secret = "aspsAFfrKJ"
    encText = urllib.parse.quote(add)
    url = "https://openapi.naver.com/v1/map/geocode?encoding=utf-8&coordType=latlng&query=" + encText
    request = urllib.request.Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    response = urllib.request.urlopen(request)
    response_body = response.read()
    list = json.loads(response_body)
    return list['result']['items'][0]['point']

In [18]:
trans_coords(trans_adds(df['mapx'][69],df['mapy'][69]))

{'x': 126.9846894, 'y': 37.5620062}

In [19]:
KakaoAK = ["KakaoAK 5195aa628fe0a657648118172db5a611","KakaoAK 0b57d37a61f9b28a8b30819c7071d1ce","KakaoAK 2912bb5a8577c30a55c3fd67c64a9975",
          "KakaoAK 62db5476f97ff8c2e62f85355d0fb617","KakaoAK c91c5684a0fe4d3c22af518a226f44a1"]
def around_place(x,y,category,i=0):
    url_base = "https://dapi.kakao.com/v2/local/search/category.json?radius=500"
    url_x = "&x=" + str(x)
    url_y = "&y=" + str(y)
    url_category = "&category_group_code=" + category
    url = url_base + url_x + url_y + url_category
    
    request = urllib.request.Request(url)

    request.add_header("Authorization",KakaoAK[i])
    response = urllib.request.urlopen(request)
    resbody = response.read()
    list = json.loads(resbody)
    return list['meta']['total_count']

In [20]:
categories = {'대형마트':'MT1','편의점':'CS2','어린이집,유치원':'PS3','학교':'SC4','학원':'AC5','주차장':'PK6','주유소':'OL7',
            '지하철역':'SW8','은행':'BK9','문화시설':'CT1','중개업소':'AG2','공공기관':'PO3','관광명소':'AT4','숙박':'AD5',
              '음식점':'FD6','카페':'CE7','병원':'HP8','약국':'PM9'}

In [21]:
for name, code in categories.items():
    df[name]=''

In [22]:
categories['은행']

'BK9'

In [25]:
df['x'] = ''
df['y'] = ''
j = 0
for i in tqdm_notebook(range(len(df[9720:]))):
    try:
        df['x'][i+9720] = trans_coords(trans_adds(df['mapx'][i+9720],df['mapy'][i+9720]))['x']
        df['y'][i+9720] = trans_coords(trans_adds(df['mapx'][i+9720],df['mapy'][i+9720]))['y']
#         try:
#             for name, code in categories.items():
#                 df[name][i+9720] = around_place(df['x'][i+9720],df['y'][i+9720],code,j)
#                 j += 1
#                 if j == 5:
#                     j = 0
#         except :
#             print(i+9720)
#             pass

    except :
        print(i)
        pass
#     5882
#     6212

C:\Users\ce401Prof\bigdata\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\ce401Prof\bigdata\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\ce401Prof\bigdata\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.


1188
2134
13420
14179


In [27]:
df.to_excel('./data/dataplc.xlsx')